<a href="https://colab.research.google.com/github/tilaboy/nlp_transformer_tutorial/blob/main/learning_notes/ch5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets --quiet
!pip install transformers --quiet
!pip install pandas --quiet
!pip install numpy --quiet
!pip install torch --quiet

In [ ]:
import torch 
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import numpy as np

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu" 
model_name = "gpt2-small" 
tokenizer = AutoTokenizer.from_pretrained(model_name) 
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

OSError: ignored

In [ ]:
input_txt = "Transformers are the" 
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device) 
iterations = [] 
n_steps = 8 
choices_per_step = 5
with torch.no_grad(): 
    for _ in range(n_steps): 
        iteration = dict() 
        iteration["Input"] = tokenizer.decode(input_ids[0]) 
        output = model(input_ids=input_ids) 
        # Select logits of the first batch and the last token and apply softmax
        next_token_logits = output.logits[0, -1, :]
        next_token_probs = torch.softmax(next_token_logits, dim=-1) 
        sorted_ids = torch.argsort(next_token_probs, dim=-1, descending=True) 
        # Store tokens with highest probabilities 
        for choice_idx in range(choices_per_step): 
            token_id = sorted_ids[choice_idx] 
            token_prob = next_token_probs[token_id].cpu().numpy() 
            token_choice = ( f"{tokenizer.decode(token_id)} ({100 * token_prob:.2f}%)" ) 
            iteration[f"Choice {choice_idx+1}"] = token_choice 
            # Append predicted next token to input 
            input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim=-1) 
            iterations.append(iteration) pd.DataFrame(iterations)
pd.DataFrame(iterations)
